In [1]:
! pip install datasets evaluate transformers rouge-score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 467.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=577cad702de38ecff06d26a21d2e12bc900067c561480752630453c2b8198e1b
  Stored in directory: /Users/herooooooooo/Library/Caches/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [4]:
import transformers

print(transformers.__version__)

4.37.2


In [5]:
model_checkpoint = "t5-small"

In [6]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("xsum")
metric = load("rouge")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [7]:
raw_datasets["test"][0].keys()

dict_keys(['document', 'summary', 'id'])

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,"Australia, the reigning champions in both competitions, will host the events, with the finals being played back-to-back on 2 December in Brisbane.\nAustralia, England, Canada, New Zealand and Papua New Guinea are already sure of places at the women's tournament.\nFiji, Tonga, Samoa and the Cook Islands will compete for the final place in the Pacific qualifying series in October.\nThe tournament's CEO, Andrew Hill, said it was a ""game-changer"" for the sport.\n""It has always been the board's vision to have the Women's Rugby League World Cup played concurrently with the men's tournament,"" he added.\n""We have been able to create what is a milestone moment for the sport.""",The 2017 Women's Rugby League World Cup will be held alongside the men's tournament for the very first time.,37348745
1,"Enda Kenny, Taoiseach (Irish prime minister) and leader of the largest coalition party, Fine Gael, said it was a disappointing election for his party.\nIt is likely Fine Gael will remain the largest party, but with a narrow lead over the main opposition, Fianna Fáil.\nThere is growing pressure on the two parties to provide a government.\nBefore the election, both of them indicated that they would not go into coalition with each other.\nCounting of ballots will continue on Sunday, with the possibility that some seats may not be declared until Monday.\nWhile existing coalition partners Fine Gael and Labour have suffered heavy losses, Fianna Fáil, Sinn Féin, smaller parties and Independents are doing well.\nWith 95 seats filled, Fine Gael has 28 seats; Labour four; Fianna Fáil 28; Sinn Féin 13; Anti-Austerity-Alliance-People Before Profit four; the Social Democrats three; the Green Party one; the Independent Alliance four and 10 seats have gone to Independents.\nOn Saturday, Mr Kenny - who has been re-elected in Mayo - said it was clear the existing government would not regain power.\n""This is a disappointment for the Fine Gael party,"" he said.\nHe said you can ""argue about"" the people's decision, but ""you cannot argue with it"".\n""So clearly, the government of Fine Gael and Labour are not going to be returned to office.""\nHe added that the ""majority government option is gone"" but he would wait for the full results and consider options on how to form a government ""very carefully"".\nFianna Fáil leader Micheál Martin described the election as an ""extraordinary vote"" and said he was ""very pleased"".\nHe said it would take time before it was clear what shape a new government would take.\n""A lot will depend on the destination of the last seats in many constituencies. It's a bit too early to be definitive, but it's clear we're going to have a good day,"" he said.\nJoan Burton said she was ""very, very disappointed"" that many Labour Party candidates would not be elected to parliament.\n""All I can say is that the Labour Party is at the fight for justice and social justice in Ireland and we will continue that fight in the next Dáil, even if our numbers are diminished.""\nAmong the other notable candidates elected so far are Sinn Féin deputy leader Mary-Lou McDonald and Fine Gael ministers Frances Fitzgerald, Charlie Flanagan and Leo Varadkar.\nHowever, Fine Gael's former Justice Minister Alan Shatter lost his seat, as has Alex White, the Labour Party's communications minister. Children's Minister James Reilly looks likely to lose his too.\nSinn Féin, another big winner, indicated it would not go into government.\nThe party's Martin McGuinness, Northern Ireland's deputy first minister, said it had been a hugely successful election for Sinn Féin.\n""My sense is that a government will be formed, I don't think there will be a second election within weeks. How stable a government, that remains to be seen,"" he said.\nOver three million people were entitled to vote in Friday's poll, which will return 157 members of parliament, known as TDs. The ceann comhairle (speaker) is automatically returned.\nThe campaign was f

In [12]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [14]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [15]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [18]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or NPU devices or certain XPU devices (with IPEX).